# Introdução ao Spark com Python

## RDD: `sample` e `takeSample`

## Como obter o `SparkContext`

In [2]:
import pyspark
sc = pyspark.SparkContext(appName="P3")
sc

<SparkContext master=local[*] appName=P3>

## Obtendo o conjunto de dados de análise completo

Usaremos um conjunto dados completo da Copa KDD de 1999, que contém quase meio milhão de registros. O arquivo é fornecido como um *Gzip*.

In [3]:
import urllib.request as request
f = request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")

Usaremos este arquivos para criar nosso RDD

In [4]:
nome_arquivo = "./kddcup.data.gz"
rdd = sc.textFile(nome_arquivo)

## Amostrando RDDs   

No Spark, existem duas operações de amostragem: a transformação `sample` e a ação `takeSample`.

### A transformação `sample`

A transformação `sample` aceita até 3 parâmetros. O primeiro trata de informar se a amostragem é feita com substituição ou não. O segundo é o tamanho da amostra como fração. O último parâmetro informa um *random seed*.  

In [6]:
amostra_rdd = rdd.sample(False, 0.1, 1234)
tamanho_amostra = amostra_rdd.count()
tamanho_total = rdd.count()
print("O tamanho da amostra é {} de {}".format(tamanho_amostra, tamanho_total))

O tamanho da amostra é 489957 de 4898431


Imagine que desejemos ter uma aproximação da proporção de interações `normais` em nosso conjunto de dados. Poderíamos realizar a tarefa contando o número de instâncias com a palavra normal como anteriormente. 

Para uma resposta mais rápida, podemos fazer como a seguir:

In [7]:
from time import time

# transformações a serem aplicadas
itens_amostra_rdd = amostra_rdd.map(lambda x: x.split(","))
itens_amostra_normais = itens_amostra_rdd.filter(lambda x: "normal." in x)

# ações + tempo
t0 = time()
contagem_itens_amostra_normais = itens_amostra_normais.count()
tt = time() - t0

razao_amostra_normal = contagem_itens_amostra_normais / float(tamanho_amostra)
print("A proporção de interações 'normais' é {}".format(round(razao_amostra_normal,3)) )
print("Contagem realizada em {} segundos".format(round(tt,3)))

A proporção de interações 'normais' é 0.199
Contagem realizada em 7.949 segundos


Compare com o cálculo da razão sem a amostragem

In [8]:
# transformações a serem aplicadas
itens_rdd = rdd.map(lambda x: x.split(","))
itens_normais = itens_rdd.filter(lambda x: "normal." in x)

# ações + tempo
t0 = time()
contagem_itens_normais = itens_normais.count()
tt = time() - t0

razao_normal = contagem_itens_normais / float(tamanho_total)
print("A proporção de interações 'normais' é {}".format(round(razao_normal,3)) )
print("Contagem realizada em {} segundos".format(round(tt,3)))

A proporção de interações 'normais' é 0.199
Contagem realizada em 16.559 segundos


Demora muito mais. Isso porque sem a amostragem todas as transformações são aplicadas ao conjunto completo de dados.

### A ação `takeSample`

Usamos a ação `takeSample` para amostrar nosso RDD para ser utilizado por bibliotecas não-spark.

A sintaxe é bem similar, mas neste caso especificamos o número de itens ao invés do tamanho da amostra como uma fração do conjunto completo.

In [11]:
t0 = time()
amostra_rdd = rdd.takeSample(False, 400000, 1234)
amostra_rdd_normal = [x.split(",") for x in amostra_rdd if "normal." in x]
tt = time() - t0

tamanho_amostra_normal = len(amostra_rdd_normal)

razao_normal = tamanho_amostra_normal / 400000.0
print("A proporção das interações 'normais' é {}".format(razao_normal))
print("Contagem realizada em {} segundos".format(round(tt,3)))

A proporção das interações 'normais' é 0.1988025
Contagem realizada em 14.542 segundos


O processo foi similar ao anterior. Obtivemos uma amostra de cerca de 10% dos dados, filtramos e aplicamos um split.

No entanto, demorou mais, mesmo com uma amostra ligeiramente menor. Isso porque o Spark distribuiu a execução do processo de amostragem. A filtragem e o splitting dos resultados foram feitos localmente em um único nó.